<div align="center">
  
# 🎮 Laboratorio 7: Poke-ensambles ⚡

<img src="https://99px.ru/sstorage/86/2016/11/image_862911161940187734278.gif" width="250"/>

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; margin: 20px 0;">
  <h3 style="color: white; margin: 0;">💤 Team Snorlax 💤</h3>
  <p style="color: white; font-style: italic;">"Parece que dormimos, pero nuestros modelos están entrenando"</p>
</div>

</div>

---

**Duración:** 2 horas  
**Formato:** Competencia por equipos con rotaciones y
kahoots Pokemon  
**Repositorio base:** [_link al repo oficial_](https://github.com/JAlbertoAlonso/poke-ensambles/tree/main)

---

<div style="background-color: #f8f9fa; padding: 15px; border-left: 5px solid #667eea; border-radius: 5px;">

### 👥 Integrantes del Equipo

- 🌊 Melisa Asharet Arano Bejarano (@Asharetbe)
- 🧢 Erick José Fabián Sandoval
- 🪨 Abril Minerva Estrada Montaño
- 🔥 Sebastián Alanis Gónzalez
- 🌱 Jose Daniel Vera Garfias

**Repositorio del equipo:**  
<https://github.com/usuario/equipoX>

**Fecha de entrega:**  
__/__/____

In [1]:
# Paqueterías
import csv, math, random, time, requests
from collections import defaultdict
import os
import re
import unicodedata
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Modelos PERMITIDOS (base, no ensambles)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


<div align="center">
  
# 🗺️ Elemento 1 - Construcción del dataset

<img src="https://i.pinimg.com/originals/3b/22/a9/3b22a9588938ea55dbc7cd4fa0f87951.gif" width="300"/>

<p style="font-style: italic; color: #666;">Capturando datos de Pokémon desde la PokéAPI</p>

</div>

---

<div style="background: linear-gradient(to right, #f093fb 0%, #f5576c 100%); padding: 2px; border-radius: 10px; margin: 20px 0;">
  <div style="background: white; padding: 20px; border-radius: 8px;">
    <h2 style="color: #f5576c; margin-top: 0;">🌐 Fuentes de datos</h2>
    <p>Utilizamos la PokéAPI, una base de datos pública con información de todos los Pokémon:</p>
    <p>📡 Endpoint principal: <a href="https://pokeapi.co/api/v2/pokemon/">https://pokeapi.co/api/v2/pokemon/</a></p>
    <p>Ejemplo de consulta:<br>
    <a href="https://pokeapi.co/api/v2/pokemon/1">https://pokeapi.co/api/v2/pokemon/1</a> → Devuelve información de Bulbasaur</p>
  </div>
</div>

---

## 📊 Variables del modelo

<table style="width: 100%; border-collapse: collapse;">
  <tr style="background-color: #667eea; color: white;">
    <th style="padding: 12px; text-align: left;">Categoría</th>
    <th style="padding: 12px; text-align: left;">Variables</th>
    <th style="padding: 12px; text-align: left;">Descripción</th>
  </tr>
  <tr style="background-color: #f8f9fa;">
    <td style="padding: 12px;">Features (visibles)</td>
    <td style="padding: 12px;">
      <code>height</code><br>
      <code>weight</code><br>
      <code>base_experience</code><br>
      <code>type_main</code>
    </td>
    <td style="padding: 12px;">
      Altura del Pokémon<br>
      Peso del Pokémon<br>
      Experiencia base<br>
      Tipo principal (Fuego, Agua, etc.)
    </td>
  </tr>
  <tr>
    <td style="padding: 12px;">Stats base (para etiqueta)</td>
    <td style="padding: 12px;">
      <code>hp</code>, <code>attack</code>, <code>defense</code><br>
      <code>special_attack</code>, <code>special_defense</code><br>
      <code>speed</code>
    </td>
    <td style="padding: 12px;">
      Estadísticas de combate<br>
      usadas solo para calcular<br>
      la etiqueta <code>strong</code>
    </td>
  </tr>
</table>

---

<div style="background-color: #fff3cd; border-left: 5px solid #ffc107; padding: 15px; border-radius: 5px; margin: 20px 0;">
  
## ⚡ Definición de la etiqueta binaria

Calculamos el puntaje de poder total sumando todas las estadísticas base:

$$
power\_score = hp + attack + defense + special\_attack + special\_defense + speed
$$

La etiqueta binaria <code>strong</code> se define como:

$$
strong =
\begin{cases}
1, & \text{si } power\_score > P_{75}(power\_score) \\
0, & \text{en otro caso}
\end{cases}
$$

</div>

---

## 📋 Dataset final

<div style="overflow-x: auto;">
  
<table style="width: 100%; border-collapse: collapse; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
  <thead>
    <tr style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white;">
      <th style="padding: 12px;">Nº</th>
      <th style="padding: 12px;">Variable</th>
      <th style="padding: 12px;">Descripción</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">1</td><td style="padding: 10px;"><code>name</code></td><td style="padding: 10px;">Nombre del Pokémon</td></tr>
    <tr><td style="padding: 10px;">2</td><td style="padding: 10px;"><code>height</code></td><td style="padding: 10px;">Altura</td></tr>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">3</td><td style="padding: 10px;"><code>weight</code></td><td style="padding: 10px;">Peso</td></tr>
    <tr><td style="padding: 10px;">4</td><td style="padding: 10px;"><code>base_experience</code></td><td style="padding: 10px;">Experiencia base</td></tr>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">5</td><td style="padding: 10px;"><code>type_main</code></td><td style="padding: 10px;">Tipo principal</td></tr>
    <tr><td style="padding: 10px;">6</td><td style="padding: 10px;"><code>hp</code></td><td style="padding: 10px;">Puntos de vida</td></tr>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">7</td><td style="padding: 10px;"><code>attack</code></td><td style="padding: 10px;">Ataque</td></tr>
    <tr><td style="padding: 10px;">8</td><td style="padding: 10px;"><code>defense</code></td><td style="padding: 10px;">Defensa</td></tr>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">9</td><td style="padding: 10px;"><code>special_attack</code></td><td style="padding: 10px;">Ataque especial</td></tr>
    <tr><td style="padding: 10px;">10</td><td style="padding: 10px;"><code>special_defense</code></td><td style="padding: 10px;">Defensa especial</td></tr>
    <tr style="background-color: #f8f9fa;"><td style="padding: 10px;">11</td><td style="padding: 10px;"><code>speed</code></td><td style="padding: 10px;">Velocidad</td></tr>
    <tr><td style="padding: 10px;">12</td><td style="padding: 10px;"><code>power_score</code></td><td style="padding: 10px;">Suma de stats base</td></tr>
    <tr style="background-color: #ffe0e0;"><td style="padding: 10px;">13</td><td style="padding: 10px;"><code>strong</code></td><td style="padding: 10px;">Etiqueta binaria</td></tr>
  </tbody>
</table>

</div>

---

<div align="center">
  <img src="https://i.gifer.com/embedded/download/C7SV.gif" width="200"/>
</div>

## 🎯 Muestreo estratificado

Para obtener una muestra representativa de 500 Pokémon, implementamos muestreo estratificado por:

- <code>type_main</code> - Tipo principal del Pokémon
- <code>strong</code> - Nivel de poder (fuerte vs regular)

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 10px; margin: 15px 0;">

### 🎲 Proceso de muestreo

```
1. Agrupar población por (type_main, strong)
2. Calcular proporción de cada estrato
3. Asignar cuota proporcional (≥ 1 por estrato)
4. Ajustar para obtener exactamente 500 Pokémon
5. Seleccionar aleatoriamente dentro de cada estrato
```

</div>

---

In [2]:
# Configuración
API_ROOT = "https://pokeapi.co/api/v2"
TARGET_SAMPLE = 500
RANDOM_SEED = 42

In [3]:
def percentile(values, p):
  """
  Calcula el percentil p (entre 0 y 1) de una lista, ignorando None
  """
  xs = sorted(v for v in values if v is not None)
  if not xs: return float("nan")
  idx = (len(xs) - 1) * p
  lo, hi = math.floor(idx), math.ceil(idx)
  if lo == hi: return xs[lo]
  w = idx - lo
  return xs[lo]*(1-w) + xs[hi]*w

def fetch_json(url, retries=4, base_sleep=0.6, skip_404=True):

  """
  Hace un GET a url y devuelve el JSON con reintentos
  """
  for attempt in range(retries):
      try:
          r = requests.get(url, timeout=30)
          if skip_404 and r.status_code == 404:
              return None
          r.raise_for_status()
          return r.json()
      except Exception:
          if attempt == retries - 1:
              raise
          time.sleep(base_sleep * (attempt + 1))


def get_stat(pkm, name):
  """
  Obtiene el valor de una estadística de un Pokémon
  """
  for s in pkm.get("stats", []):
      if s.get("stat", {}).get("name") == name:
          return s.get("base_stat")
  return None


def main_type(pkm):
  """
  Obtiene el tipo principal de un Pokémon
  """
  types = list(pkm.get("types") or [])
  if not types: return None
  types.sort(key=lambda t: t.get("slot", 99))
  return types[0].get("type", {}).get("name")


def list_all_pokemon_names():
  """
  Obtiene una lista con los nombres de todos los Pokémon
  """
  names = []
  url = f"{API_ROOT}/pokemon?limit=20000&offset=0"
  while url:
      j = fetch_json(url, skip_404=False)
      names.extend([it["name"] for it in j.get("results", [])])
      url = j.get("next")
      time.sleep(0.15)  # rate limit
  return names

def build_full_population():
  """
  Construye la población completa de Pokémon
  """
  names = list_all_pokemon_names()
  rows, kept = [], 0
  for i, name in enumerate(names, 1):
      pkm = fetch_json(f"{API_ROOT}/pokemon/{name}", skip_404=True)
      if pkm is None:
          continue

      if not pkm.get("is_default", True):
          continue

      hp       = get_stat(pkm, "hp")
      attack   = get_stat(pkm, "attack")
      defense  = get_stat(pkm, "defense")
      sp_atk   = get_stat(pkm, "special-attack")
      sp_def   = get_stat(pkm, "special-defense")
      speed    = get_stat(pkm, "speed")
      pscore   = sum(v or 0 for v in [hp, attack, defense, sp_atk, sp_def, speed])

      rows.append({
          "name": pkm.get("name"),
          "height": pkm.get("height"),
          "weight": pkm.get("weight"),
          "base_experience": pkm.get("base_experience"),
          "type_main": main_type(pkm),
          "hp": hp, "attack": attack, "defense": defense,
          "special_attack": sp_atk, "special_defense": sp_def, "speed": speed,
          "power_score": pscore,
      })
      kept += 1
      if kept % 50 == 0:
          print(f"Descargados: {i} | Población 'default' acumulada: {kept}")
      time.sleep(0.12)

  # Etiqueta strong por P75 global
  p75 = percentile([r["power_score"] for r in rows], 0.75)
  for r in rows:
      r["strong"] = int(r["power_score"] > p75)
  return rows

def stratified_sample(rows, n=TARGET_SAMPLE, seed=RANDOM_SEED):
    random.seed(seed)
    strata = defaultdict(list)
    for r in rows:
        strata[(r.get("type_main"), r.get("strong"))].append(r)
    N = sum(len(v) for v in strata.values())
    if N == 0: raise RuntimeError("Población vacía.")
    quotas, fracs, tot = {}, [], 0
    for k, items in strata.items():
        Nh = len(items)
        raw = n * (Nh / N)
        q = max(1, int(math.floor(raw)))
        quotas[k] = q; tot += q; fracs.append((k, raw - math.floor(raw)))
    if tot > n:
        fracs.sort(key=lambda x: x[1])
        i = 0
        while tot > n and i < len(fracs):
            k, _ = fracs[i]
            if quotas[k] > 1:
                quotas[k] -= 1; tot -= 1
            i += 1
    elif tot < n:
        fracs.sort(key=lambda x: x[1], reverse=True)
        i = 0
        while tot < n and i < len(fracs):
            k, _ = fracs[i]
            if quotas[k] < len(strata[k]):
                quotas[k] += 1; tot += 1
            i += 1
    sample = []
    for k, items in strata.items():
        q = min(quotas[k], len(items))
        sample.extend(random.sample(items, q))
    if len(sample) > n:
        sample = random.sample(sample, n)
    return sample

def write_csv(path, rows):
  """
  Escribe un archivo CSV con las filas en rows
  """
  headers = [
      "name","height","weight","base_experience","type_main",
      "hp","attack","defense","special_attack","special_defense","speed",
      "power_score","strong"
  ]
  with open(path, "w", newline="", encoding="utf-8") as f:
      w = csv.DictWriter(f, fieldnames=headers)
      w.writeheader()
      for r in rows:
          w.writerow({h: r.get(h) for h in headers})

def main():
  """
  Genera el proceso completo de descarga y procesamiento de datos
  """
  print("Descargando población completa")
  rows = build_full_population()
  print(f"Población limpia: {len(rows)}")

  write_csv("pokemon_full.csv", rows)
  print("Guardado: pokemon_full.csv")

  sample = stratified_sample(rows, n=TARGET_SAMPLE)
  write_csv("pokemon_sample_500.csv", sample)
  print("Guardado: pokemon_sample_500.csv")

  # resumen de estratos
  dist_full, dist_smpl = defaultdict(int), defaultdict(int)
  for r in rows:   dist_full[(r["type_main"], r["strong"])] += 1
  for r in sample: dist_smpl[(r["type_main"], r["strong"])] += 1
  print("\n(type_main, strong): población -> muestra")
  for k in sorted(dist_full, key=lambda k: (str(k[0]), k[1])):
      print(f"{k}: {dist_full[k]} -> {dist_smpl.get(k, 0)}")

if __name__ == "__main__":
  main()

Descargando población completa
Descargados: 50 | Población 'default' acumulada: 50
Descargados: 100 | Población 'default' acumulada: 100
Descargados: 150 | Población 'default' acumulada: 150
Descargados: 200 | Población 'default' acumulada: 200
Descargados: 250 | Población 'default' acumulada: 250
Descargados: 300 | Población 'default' acumulada: 300
Descargados: 350 | Población 'default' acumulada: 350
Descargados: 400 | Población 'default' acumulada: 400
Descargados: 450 | Población 'default' acumulada: 450
Descargados: 500 | Población 'default' acumulada: 500
Descargados: 550 | Población 'default' acumulada: 550
Descargados: 600 | Población 'default' acumulada: 600
Descargados: 650 | Población 'default' acumulada: 650
Descargados: 700 | Población 'default' acumulada: 700
Descargados: 750 | Población 'default' acumulada: 750
Descargados: 800 | Población 'default' acumulada: 800
Descargados: 850 | Población 'default' acumulada: 850
Descargados: 900 | Población 'default' acumulada: 900

<div align="center">
  
# ⚔️ Elemento 2 - Modelos base

<img src="https://i.pinimg.com/originals/02/52/0b/02520bafe50166360f69ac620b22f614.gif" width="350"/>

<p style="font-style: italic; color: #666;">¡Entrenando nuestro equipo de modelos!</p>

</div>

---

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">
    <h2 style="color: #f5576c; margin-top: 0;">🎯 Objetivo</h2>
    <p style="font-size: 16px; line-height: 1.6;">
      Entrenar 9 modelos de clasificación supervisada usando únicamente las características visibles 
      (<code>height</code>, <code>weight</code>, <code>base_experience</code>, <code>type_main</code>) 
      para predecir la etiqueta binaria <code>strong</code>.
    </p>
  </div>
</div>

---

## 🤖 Los 9 modelos entrenados

<div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(280px, 1fr)); gap: 15px; margin: 20px 0;">

<div style="background-color: #e3f2fd; border-left: 5px solid #2196f3; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #2196f3;">1️⃣ KNN (K-Nearest Neighbors)</h3>
  <p>Estrategia: Clasificación por vecindad (k=7)</p>
</div>

<div style="background-color: #f3e5f5; border-left: 5px solid #9c27b0; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #9c27b0;">2️⃣ Logistic Regression</h3>
  <p>Estrategia: Frontera lineal probabilística</p>
</div>

<div style="background-color: #e8f5e9; border-left: 5px solid #4caf50; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #4caf50;">3️⃣ Decision Tree</h3>
  <p>Estrategia: Árbol de decisión (max_depth=6)</p>
</div>

<div style="background-color: #fff3e0; border-left: 5px solid #ff9800; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #ff9800;">4️⃣ SVM (Support Vector Machine)</h3>
  <p>Estrategia: Máximo margen de separación (kernel RBF)</p>
</div>

<div style="background-color: #fce4ec; border-left: 5px solid #e91e63; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #e91e63;">5️⃣ Linear SVC</h3>
  <p>Estrategia: SVM lineal (más rápido)</p>
</div>

<div style="background-color: #e0f2f1; border-left: 5px solid #009688; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #009688;">6️⃣ SGD Classifier</h3>
  <p>Estrategia: Descenso de gradiente estocástico</p>
</div>

<div style="background-color: #f1f8e9; border-left: 5px solid #8bc34a; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #8bc34a;">7️⃣ Ridge Classifier</h3>
  <p>Estrategia: Regresión regularizada L2</p>
</div>

<div style="background-color: #ede7f6; border-left: 5px solid #673ab7; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #673ab7;">8️⃣ LDA (Linear Discriminant Analysis)</h3>
  <p>Estrategia: Proyección discriminante</p>
</div>

<div style="background-color: #ffebee; border-left: 5px solid #f44336; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #f44336;">9️⃣ Gaussian Naive Bayes</h3>
  <p>Estrategia: Probabilidad bayesiana</p>
</div>

</div>

---

## 📈 Proceso de entrenamiento

<div style="background-color: #fff8e1; padding: 20px; border-radius: 10px; border: 2px dashed #ffc107; margin: 20px 0;">

### 🔄 Pipeline estándar

```
1. Dividir dataset → Train (80%) / Test (20%)
2. Preprocesar datos:
   • Imputación de valores faltantes (SimpleImputer)
   • Estandarización de numéricas (StandardScaler)
   • One-hot encoding para categóricas (OneHotEncoder)
3. Entrenar modelo usando SOLO train set
4. Evaluar en test set
5. Registrar métricas (accuracy)
6. Cross-validation 3-fold para robustez
```

Los modelos NO ven las stats base durante el entrenamiento. Solo pueden usar <code>height</code>, <code>weight</code>, <code>base_experience</code> y <code>type_main</code>.

</div>

---

<div style="background: linear-gradient(to right, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 12px; margin: 25px 0;">
  <h3 style="margin-top: 0;">💡 ¿Por qué 9 modelos diferentes?</h3>
  <p style="line-height: 1.7;">
    Cada algoritmo tiene sus fortalezas y debilidades. Al entrenar múltiples modelos, podemos:
  </p>
  <ul style="line-height: 1.8;">
    <li>✅ Comparar diferentes estrategias de clasificación</li>
    <li>✅ Identificar qué modelos funcionan mejor con nuestros datos</li>
    <li>✅ Combinarlos en ensambles para mejorar el rendimiento</li>
    <li>✅ Reducir el riesgo de overfitting a un modelo específico</li>
  </ul>
</div>

---

<div align="center">
  <img src="https://i.gifer.com/embedded/download/C7SV.gif" width="180"/>
  <p style="font-style: italic; color: #999; margin-top: 10px;">¡Listos para el siguiente nivel: Ensambles! 🚀</p>
</div>


In [9]:
# Conclusiones

print("\n CONCLUSIONES")
# 1) Mejor(es) modelos
sorted_holdout = sorted(accs.items(), key=lambda x: x[1], reverse=True)
best_name, best_acc = sorted_holdout[0]
print(f"- Mejor en holdout: {best_name} (accuracy={best_acc:.4f}).")

if cv_scores:
    best_cv = sorted(cv_scores.items(), key=lambda x: x[1][0], reverse=True)[0]
    print(f"- Mejor en CV 3-fold: {best_cv[0]} (mean={best_cv[1][0]:.4f}, std={best_cv[1][1]:.4f}).")

# 2) Señales de las variables
num_influential = sorted(corrs.items(), key=lambda kv: abs(kv[1]) if kv[1]==kv[1] else -1, reverse=True)
if num_influential:
    top_num = [f"{k} ({v:+.3f})" for k,v in num_influential]
    print(f"- Señal numérica (|corr| con 'strong'): " + ", ".join(top_num))

# 3) Tipos más asociados a 'strong'
if not type_rates.empty:
    top_types = type_rates.head(5).index.tolist()
    print(f"- Tipos con mayor tasa de 'strong' (Top 5): {top_types}")

# 4) Balance de clases
minority = int(class_counts.idxmin())
minority_pct = class_counts.min() / len(y) * 100
print(f"- La clase minoritaria es '{minority}' con {minority_pct:.1f}%: considera estratificación (ya aplicada) y métricas adicionales si estuviera muy desbalanceado.")



 CONCLUSIONES
- Mejor en holdout: tree (accuracy=0.9700).
- Mejor en CV 3-fold: tree (mean=0.9420, std=0.0171).
- Señal numérica (|corr| con 'strong'): base experience (+0.709), height (+0.493), weight (+0.407)
- Tipos con mayor tasa de 'strong' (Top 5): ['dragon', 'steel', 'dark', 'ice', 'flying']
- La clase minoritaria es '1' con 24.8%: considera estratificación (ya aplicada) y métricas adicionales si estuviera muy desbalanceado.


In [4]:
# Configuración

PATH_TRAIN = "pokemon_sample_500.csv"
PATH_TEST  = None  # ej. "data/testset.csv" si lo tienes

FEATURES = ["height", "weight", "base experience", "type main"]
TARGET   = "strong"
RANDOM_STATE = 42

# Funciones

def _norm(s: str) -> str:
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_")

def safe_point_biserial(x: np.ndarray, y: np.ndarray) -> float:
    # correlación simple entre binaria y numérica (equivalente a Pearson con binaria)
    try:
        if x.ndim > 1:
            x = x.ravel()
        if np.std(x) == 0 or np.std(y) == 0:
            return np.nan
        return np.corrcoef(x, y)[0,1]
    except Exception:
        return np.nan

def find_col(df: pd.DataFrame, key: str) -> str | None:
    norm_map = {_norm(c): c for c in df.columns}
    # Define ALIASES here
    ALIASES = {
        "height": ["height", "altura"],
        "weight": ["weight", "peso"],
        "base experience": ["base experience", "baseexperience", "experiencia base"],
        "type main": ["type main", "typemain", "tipo principal"],
        "strong": ["strong", "fuerte"]
    }
    for alias in ALIASES.get(key, [key]):
        a = _norm(alias)
        if a in norm_map:
            return norm_map[a]
    return None

In [5]:
# Datos

df = pd.read_csv(PATH_TRAIN)
orig_cols = list(df.columns)

mapped = {}
for k in FEATURES + [TARGET]:
    col = find_col(df, k)

    # Respaldo para type_main: si hay columna que contenga "type" con lista de tipos, toma el 1er tipo
    if k == "type main" and col is None:
        candidates = [c for c in orig_cols if "type" in _norm(c)]
        if candidates:
            col_types = candidates[0]
            df["__type_main__"] = (
                df[col_types].astype(str)
                .str.replace(r"[\[\]\{\}\'\"]", "", regex=True)
                .str.split(r"[,\s]+", regex=True)
                .apply(lambda xs: xs[0].lower() if xs and xs[0] not in [None, "nan", ""] else None)
            )
            col = "__type_main__"

    mapped[k] = col

missing = [k for k, v in mapped.items() if v is None]
if missing:
    print("\n[ERROR] No se encontraron columnas requeridas:", missing)
    print("Columnas en tu archivo:")
    for c in orig_cols:
        print(" -", c)
    raise SystemExit(1)

X = df[[mapped["height"], mapped["weight"], mapped["base experience"], mapped["type main"]]].copy()
X.columns = FEATURES
y = df[mapped[TARGET]].astype(int).values

# Exploración de datos

print("\n INFORMACIÓN DEL DATASET")
print(f"Filas: {len(df):,}")
print("Balance de clases (strong):")
class_counts = pd.Series(y).value_counts().sort_index()
for c in class_counts.index:
    pct = class_counts[c] / len(y) * 100
    print(f"  {c}: {class_counts[c]} ({pct:.1f}%)")

# Estadísticas numéricas
num_summary = pd.DataFrame({
    "mean": X[["height","weight","base experience"]].mean(),
    "std":  X[["height","weight","base experience"]].std(),
    "min":  X[["height","weight","base experience"]].min(),
    "max":  X[["height","weight","base experience"]].max(),
})
print("\nEstadísticas numéricas")
print(num_summary.round(2))

# Correlación simple con el target
corrs = {
    col: safe_point_biserial(X[col].to_numpy(dtype=float), y)
    for col in ["height","weight","base experience"]
}
print("\nCorrelación con 'strong':")
for k,v in corrs.items():
    print(f"  {k:>16s}: {v:+.3f}")

# Tasa de strong por tipo
type_rates = X.groupby("type main")["type main"].count().to_frame("n")
type_rates["strong_rate"] = (
    df.loc[X.index, mapped[TARGET]].groupby(X["type main"]).mean()
)
type_rates = type_rates.sort_values("strong_rate", ascending=False)
print("\nTop 8 tipos por tasa de 'strong':")
print(type_rates.head(8).round(3))


 INFORMACIÓN DEL DATASET
Filas: 500
Balance de clases (strong):
  0: 376 (75.2%)
  1: 124 (24.8%)

Estadísticas numéricas
                   mean      std  min   max
height            11.68     9.35    1    92
weight           632.81  1072.50    1  9500
base experience  151.32    76.68   36   395

Correlación con 'strong':
            height: +0.493
            weight: +0.407
   base experience: +0.709

Top 8 tipos por tasa de 'strong':
            n  strong_rate
type main                 
dragon     18        0.444
steel      17        0.412
dark       22        0.409
ice        15        0.400
flying      5        0.400
fire       32        0.344
psychic    29        0.310
ground     20        0.300


In [6]:
# Preprocesamiento
num_cols = ["height", "weight", "base experience"]
cat_cols = ["type main"]

pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc", StandardScaler())
        ]), num_cols),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols),
    ],
    remainder="drop",
)

In [7]:
# Modelos

models = {
    "knn":        Pipeline([("pre", pre), ("clf", KNeighborsClassifier(n_neighbors=7))]),
    "logistic":   Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=500, random_state=RANDOM_STATE))]),
    "tree":       Pipeline([("pre", pre), ("clf", DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE))]),
    "svm_rbf":    Pipeline([("pre", pre), ("clf", SVC(kernel="rbf", probability=True, random_state=RANDOM_STATE))]),
    "linear_svc": Pipeline([("pre", pre), ("clf", LinearSVC(random_state=RANDOM_STATE))]),              # sin proba
    "sgd_log":    Pipeline([("pre", pre), ("clf", SGDClassifier(loss="log_loss", max_iter=2000, random_state=RANDOM_STATE))]),
    "ridge_cls":  Pipeline([("pre", pre), ("clf", RidgeClassifier(random_state=RANDOM_STATE))]),        # sin proba
    "lda":        Pipeline([("pre", pre), ("clf", LinearDiscriminantAnalysis())]),
    "gauss_nb":   Pipeline([("pre", pre), ("clf", GaussianNB())]),
}

In [8]:
# Entrenamiento y validación

X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

print("\nVALIDACIÓN (Holdout 80/20)")
accs = {}
for name, pipe in models.items():
    pipe.fit(X_tr, y_tr)
    y_pred = pipe.predict(X_va)
    acc = accuracy_score(y_va, y_pred)
    accs[name] = acc
    print(f"{name:>10s}: accuracy = {acc:.4f}")

# También un CV rápido para robustez (3-fold)
print("\nCV 3-fold (accuracy) — puede tardar un poco:")
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
cv_scores = {}
for name, pipe in models.items():
    try:
        scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy", n_jobs=None)
        cv_scores[name] = (scores.mean(), scores.std())
        print(f"{name:>10s}: mean={scores.mean():.4f}  std={scores.std():.4f}")
    except Exception as e:
        print(f"{name:>10s}: CV error -> {e}")


VALIDACIÓN (Holdout 80/20)
       knn: accuracy = 0.8500
  logistic: accuracy = 0.9000
      tree: accuracy = 0.9700
   svm_rbf: accuracy = 0.9000
linear_svc: accuracy = 0.9100
   sgd_log: accuracy = 0.9000
 ridge_cls: accuracy = 0.9000
       lda: accuracy = 0.9000
  gauss_nb: accuracy = 0.7300

CV 3-fold (accuracy) — puede tardar un poco:
       knn: mean=0.9000  std=0.0140
  logistic: mean=0.9200  std=0.0058
      tree: mean=0.9420  std=0.0171
   svm_rbf: mean=0.9160  std=0.0100
linear_svc: mean=0.9200  std=0.0058
   sgd_log: mean=0.8900  std=0.0150
 ridge_cls: mean=0.9180  std=0.0123
       lda: mean=0.9240  std=0.0101
  gauss_nb: mean=0.7257  std=0.1102


In [9]:
# Conclusiones

print("\n CONCLUSIONES")
# 1) Mejor(es) modelos
sorted_holdout = sorted(accs.items(), key=lambda x: x[1], reverse=True)
best_name, best_acc = sorted_holdout[0]
print(f"- Mejor en holdout: {best_name} (accuracy={best_acc:.4f}).")

if cv_scores:
    best_cv = sorted(cv_scores.items(), key=lambda x: x[1][0], reverse=True)[0]
    print(f"- Mejor en CV 3-fold: {best_cv[0]} (mean={best_cv[1][0]:.4f}, std={best_cv[1][1]:.4f}).")

# 2) Señales de las variables
num_influential = sorted(corrs.items(), key=lambda kv: abs(kv[1]) if kv[1]==kv[1] else -1, reverse=True)
if num_influential:
    top_num = [f"{k} ({v:+.3f})" for k,v in num_influential]
    print(f"- Señal numérica (|corr| con 'strong'): " + ", ".join(top_num))

# 3) Tipos más asociados a 'strong'
if not type_rates.empty:
    top_types = type_rates.head(5).index.tolist()
    print(f"- Tipos con mayor tasa de 'strong' (Top 5): {top_types}")

# 4) Balance de clases
minority = int(class_counts.idxmin())
minority_pct = class_counts.min() / len(y) * 100
print(f"- La clase minoritaria es '{minority}' con {minority_pct:.1f}%: considera estratificación (ya aplicada) y métricas adicionales si estuviera muy desbalanceado.")



 CONCLUSIONES
- Mejor en holdout: tree (accuracy=0.9700).
- Mejor en CV 3-fold: tree (mean=0.9420, std=0.0171).
- Señal numérica (|corr| con 'strong'): base experience (+0.709), height (+0.493), weight (+0.407)
- Tipos con mayor tasa de 'strong' (Top 5): ['dragon', 'steel', 'dark', 'ice', 'flying']
- La clase minoritaria es '1' con 24.8%: considera estratificación (ya aplicada) y métricas adicionales si estuviera muy desbalanceado.


## Desempeño de los modelos

Después de haber entrenado los modelos los resultados obtenidos son los siguientes:

| Modelo | Holdout Accuracy | CV Mean (3-fold) | Observaciones |
|:--|:--:|:--:|:--|
| **Decision Tree** | **0.97** | **0.942 ± 0.017** | Mejor rendimiento; puede estar capturando relaciones no lineales entre variables. |
| Logistic Regression | 0.90 | 0.920 ± 0.006 | Estable y consistente; buen desempeño para un modelo lineal. |
| Linear SVC | 0.91 | 0.920 ± 0.006 | Similar a la regresión logística, con ligera ventaja en generalización. |
| SVM (RBF) | 0.90 | 0.916 ± 0.010 | Buen equilibrio entre sesgo y varianza. |
| KNN | 0.85 | 0.900 ± 0.014 | Correcto pero sensible a la escala y vecinos. |
| Ridge / LDA / SGD | ~0.89–0.92 | Estables, sin sobreajuste. |
| GaussianNB | 0.73 | 0.726 ± 0.110 | Peor desempeño, probablemente por la no-normalidad de los datos. |

---

## Conclusiones generales

- El **árbol de decisión** obtuvo el mejor rendimiento global tanto en la validación holdout como en la validación cruzada, alcanzando una **accuracy superior al 94%**.
- Los modelos **lineales** (Logistic, Ridge, LinearSVC) mostraron **resultados consistentes y estables**, confirmando que la separación entre clases es parcialmente lineal.  
- Los modelos más simples (Naive Bayes, KNN) tuvieron desempeño inferior, evidenciando que una complejidad moderada mejora la clasificación.

En conjunto, los resultados muestran que las variables seleccionadas contienen suficiente información para distinguir entre Pokémon fuertes y débiles, siendo la **experiencia base** la característica más determinante.

<div align="center">
  
#  Elemento 3 - Ensambles de votación

<img src="https://i.gifer.com/embedded/download/C7SV.gif" width="320"/>

<p style="font-style: italic; color: #666;">¡Formando el equipo perfecto!</p>

</div>

---

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">
    <h2 style="color: #764ba2; margin-top: 0;">🎯 Concepto de Ensemble</h2>
    <p style="font-size: 16px; line-height: 1.6;">
      Los métodos de ensamble combinan las predicciones de varios clasificadores para obtener 
      decisiones más robustas y estables. La idea es que múltiples modelos diferentes 
      pueden compensar las debilidades individuales.
    </p>
  </div>
</div>

---

## 🏆 Equipo base seleccionado

<div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">

De los 9 modelos base, seleccionamos estos 4 clasificadores para formar nuestro equipo:

<div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin-top: 15px;">

<div style="background-color: #e3f2fd; border-left: 5px solid #2196f3; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #2196f3;">🎯 KNN (k=7)</h3>
  <p>Clasificación por vecindad</p>
</div>

<div style="background-color: #f3e5f5; border-left: 5px solid #9c27b0; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #9c27b0;">⚖️ Logistic Regression</h3>
  <p>Modelo lineal probabilístico</p>
</div>

<div style="background-color: #e8f5e9; border-left: 5px solid #4caf50; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #4caf50;">🌳 Decision Tree</h3>
  <p>Árbol (max_depth=6)</p>
</div>

<div style="background-color: #fff3e0; border-left: 5px solid #ff9800; padding: 15px; border-radius: 8px;">
  <h3 style="margin-top: 0; color: #ff9800;">🔍 SVM RBF</h3>
  <p>Support Vector Machine no lineal</p>
</div>

</div>
</div>

---

## 🗳️ Estrategias de votación

<div style="margin: 25px 0;">

### 1️⃣ Majority Voting (votación dura)

<div style="background-color: #fff8e1; padding: 20px; border-radius: 10px; border-left: 5px solid #ffc107; margin: 15px 0;">

Cada modelo vota 0 o 1. La clase final es la que tenga más votos:

$$
\hat{y} =
\begin{cases}
1, & \text{si } \sum_{j=1}^{M} h_j(x) > \frac{M}{2} \\
0, & \text{en otro caso}
\end{cases}
$$

Si 3 de 4 modelos predicen <code>strong=1</code>, el ensamble predice 1.

</div>

---

### 2️⃣ Soft Voting (votación suave)

<div style="background-color: #e8f5e9; padding: 20px; border-radius: 10px; border-left: 5px solid #4caf50; margin: 15px 0;">

En lugar de votos binarios, promediamos las probabilidades predichas:

$$
P(y=1|x) = \frac{1}{M} \sum_{j=1}^{M} P_j(y=1|x)
$$

Si el promedio ≥ 0.5, predecimos <code>strong=1</code>.

</div>

---

### 3️⃣ Weighted Voting (votación ponderada)

<div style="background-color: #e3f2fd; padding: 20px; border-radius: 10px; border-left: 5px solid #2196f3; margin: 15px 0;">

Similar a soft voting, pero asignamos pesos según la accuracy de cada modelo:

$$
P(y=1|x) = \frac{\sum_{j=1}^{M} w_j \cdot P_j(y=1|x)}{\sum_{j=1}^{M} w_j}
$$

donde $w_j$ es el peso (accuracy) del modelo $j$ en validación.

Los modelos más precisos tienen mayor influencia en la decisión final.

</div>

</div>

---

## 🔧 Implementación

<div style="background: linear-gradient(to right, #f093fb 0%, #f5576c 100%); padding: 2px; border-radius: 10px; margin: 20px 0;">
  <div style="background: white; padding: 20px; border-radius: 8px;">
    
### 💻 Implementación manual

Los tres votadores se implementan desde cero (sin usar <code>VotingClassifier</code> de sklearn):

```
1. Entrenar los 4 modelos base
2. Recopilar predicciones/probabilidades
3. Aplicar la estrategia de votación
4. Devolver la predicción final
```

Clases implementadas:
- <code>MajorityVoter</code> → Votación dura
- <code>SoftVoter</code> → Votación suave (promedio de probabilidades)
- <code>WeightedVoter</code> → Votación ponderada por accuracy

  </div>
</div>

---

## 📊 Evaluación

<div style="background-color: #fce4ec; padding: 20px; border-radius: 10px; margin: 20px 0;">

### 🧪 Métodos de validación

Holdout 80/20:
- 80% para entrenamiento
- 20% para validación

Cross-validation 3-fold:
- Entrena y evalúa en tres particiones estratificadas
- Obtiene media y desviación estándar
- Mide robustez y estabilidad

### 📈 Métrica

$$
\text{Accuracy} = \frac{\text{Predicciones correctas}}{\text{Total de ejemplos}}
$$

</div>


In [13]:
base_models = {
    "knn":      Pipeline([("pre", pre), ("clf", KNeighborsClassifier(n_neighbors=7))]),
    "logistic": Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=500, random_state=RANDOM_STATE))]),
    "tree":     Pipeline([("pre", pre), ("clf", DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE))]),
    "svm_rbf":  Pipeline([("pre", pre), ("clf", SVC(kernel="rbf", probability=True, random_state=RANDOM_STATE))]),
}

print("\nMODELOS BASE (holdout 80/20)")
fitted = []
val_acc = {}
for name, pipe in base_models.items():
    pipe.fit(X_tr, y_tr)
    acc = accuracy_score(y_va, pipe.predict(X_va))
    val_acc[name] = acc
    fitted.append((name, pipe))
    print(f"{name:>8s}: accuracy = {acc:.4f}")


MODELOS BASE (holdout 80/20)
     knn: accuracy = 0.8500
logistic: accuracy = 0.9000
    tree: accuracy = 0.9700
 svm_rbf: accuracy = 0.9000


In [17]:
# Clases y funciones auxiliares

class MajorityVoter:
    def __init__(self, models):
        self.models = models
    def predict(self, X):
        preds = [m.predict(X) for _, m in self.models]
        V = np.stack(preds, axis=0)
        return (V.sum(axis=0) > (len(self.models)/2)).astype(int)

class SoftVoter:
    def __init__(self, models):
        self.models = models
    def predict_proba(self, X):
        Ps = [m.predict_proba(X)[:,1] for _, m in self.models]
        return np.mean(np.stack(Ps, axis=0), axis=0)
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

class WeightedVoter:
    def __init__(self, models, weights):
        self.models = models
        w = np.asarray(weights, float)
        self.w = w / w.sum()
    def predict_proba(self, X):
        Ps = [m.predict_proba(X)[:,1] for _, m in self.models]
        P = np.stack(Ps, axis=0)
        return (self.w[:,None] * P).sum(axis=0)
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

prob_models = [(n,m) for (n,m) in fitted if hasattr(m.named_steps["clf"], "predict_proba")]
weights = [val_acc[n] for (n,_) in prob_models]

majority = MajorityVoter(fitted)
soft     = SoftVoter(prob_models)
weighted = WeightedVoter(prob_models, weights)

<div align="center">
  
# 🏁 Elemento 4 - Evaluación y competencia

<img src="https://i.gifer.com/embedded/download/8XbQ.gif" width="350"/>

<p style="font-style: italic; color: #666;">¡Que gane el mejor equipo!</p>

</div>

---

<div style="background: linear-gradient(135deg, #ff6b6b 0%, #feca57 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">
    <h2 style="color: #ff6b6b; margin-top: 0;">⚔️ La batalla final</h2>
    <p style="font-size: 16px; line-height: 1.6;">
      En esta sección evaluamos el desempeño de cada votador (Majority, Soft, Weighted) y los comparamos 
      con los modelos base individuales. El objetivo es determinar si el trabajo en equipo 
      supera al desempeño individual.
    </p>
  </div>
</div>

---

## 🎯 Métricas de evaluación

<div style="background-color: #e8f5e9; padding: 20px; border-radius: 10px; border-left: 5px solid #4caf50; margin: 20px 0;">

### Accuracy (Exactitud)

$$
\text{Accuracy} = \frac{\text{Predicciones correctas}}{\text{Total de predicciones}}
$$

Mide el porcentaje de aciertos del modelo sobre el conjunto de test.

</div>

---

## 📊 Comparación de modelos

<table style="width: 100%; border-collapse: collapse; box-shadow: 0 2px 8px rgba(0,0,0,0.1); margin: 20px 0;">
  <thead>
    <tr style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white;">
      <th style="padding: 12px; text-align: left;">Categoría</th>
      <th style="padding: 12px; text-align: left;">Modelo</th>
      <th style="padding: 12px; text-align: center;">Método de evaluación</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: #f8f9fa;">
      <td style="padding: 10px;" rowspan="4">Modelos base (individuales)</td>
      <td style="padding: 10px;">KNN</td>
      <td style="padding: 10px; text-align: center;">Holdout 80/20</td>
    </tr>
    <tr>
      <td style="padding: 10px;">Logistic Regression</td>
      <td style="padding: 10px; text-align: center;">Holdout 80/20</td>
    </tr>
    <tr style="background-color: #f8f9fa;">
      <td style="padding: 10px;">Decision Tree</td>
      <td style="padding: 10px; text-align: center;">Holdout 80/20</td>
    </tr>
    <tr>
      <td style="padding: 10px;">SVM RBF</td>
      <td style="padding: 10px; text-align: center;">Holdout 80/20</td>
    </tr>
    <tr style="background-color: #ffe0e0;">
      <td style="padding: 10px;" rowspan="3">Ensambles (votadores)</td>
      <td style="padding: 10px;">Majority Voting</td>
      <td style="padding: 10px; text-align: center;">Holdout + CV 3-fold</td>
    </tr>
    <tr style="background-color: #ffe0e0;">
      <td style="padding: 10px;">Soft Voting</td>
      <td style="padding: 10px; text-align: center;">Holdout + CV 3-fold</td>
    </tr>
    <tr style="background-color: #ffe0e0;">
      <td style="padding: 10px;">Weighted Voting</td>
      <td style="padding: 10px; text-align: center;">Holdout + CV 3-fold</td>
    </tr>
  </tbody>
</table>

---

## 🏆 ¿Quién gana?

<div style="background: linear-gradient(to right, #f093fb 0%, #f5576c 100%); padding: 2px; border-radius: 10px; margin: 20px 0;">
  <div style="background: white; padding: 20px; border-radius: 8px;">
    
###  Hipótesis

Esperamos que los ensambles superen a los modelos individuales porque:

1. Diversidad de estrategias - Cada modelo ve el problema desde un ángulo diferente
2. Compensación de errores - Los errores de un modelo pueden ser corregidos por otros
3. Mayor estabilidad - La votación reduce la varianza de las predicciones
4. Robustez - El ensamble es menos sensible a datos atípicos o ruido

  </div>
</div>

---

## 📈 Cross-Validation 3-fold

<div style="background-color: #fff3e0; padding: 20px; border-radius: 10px; border-left: 5px solid #ff9800; margin: 20px 0;">

### ¿Por qué usar CV?

La validación cruzada (Cross-Validation) nos ayuda a:

- Medir la estabilidad del modelo en diferentes particiones de datos
- Reducir el sesgo de una única partición train/test
- Obtener estadísticas (media y desviación estándar) del rendimiento

Proceso:
1. Dividir datos en 3 particiones (folds)
2. Entrenar en 2 folds, evaluar en 1
3. Repetir 3 veces (cada fold es test una vez)
4. Calcular media y desviación estándar de accuracy

</div>

---

<div align="center">
  <img src="https://i.pinimg.com/originals/3b/22/a9/3b22a9588938ea55dbc7cd4fa0f87951.gif" width="250"/>
</div>


In [16]:
# Evaluar votadores
print("\nVOTADORES")
for name, voter in [("majority", majority), ("soft", soft), ("weighted", weighted)]:
    y_hat = voter.predict(X_va)
    acc = accuracy_score(y_va, y_hat)
    print(f"{name:>8s}: accuracy = {acc:.4f}")

print("\nCV 3-fold de votadores (accuracy)")
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
for name, voter in [("majority", majority), ("soft", soft), ("weighted", weighted)]:
    scores = []
    for tr_idx, va_idx in cv.split(X, y):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y[tr_idx], y[va_idx]
        re_fitted = []
        for base_name, base_pipe in base_models.items():
            bp = base_pipe
            bp.fit(Xtr, ytr)
            re_fitted.append((base_name, bp))
        if name == "majority":
            v = MajorityVoter(re_fitted)
        else:
            pm = [(bn, bm) for (bn, bm) in re_fitted if hasattr(bm.named_steps["clf"], "predict_proba")]
            if name == "soft":
                v = SoftVoter(pm)
            else:
                w = []
                for bn, bm in pm:
                    w.append(accuracy_score(yva, bm.predict(Xva)))
                v = WeightedVoter(pm, w if sum(w) > 0 else [1]*len(pm))
        scores.append(accuracy_score(yva, v.predict(Xva)))
    print(f"{name:>8s}: mean={np.mean(scores):.4f}  std={np.std(scores):.4f}")



VOTADORES
majority: accuracy = 0.9200
    soft: accuracy = 0.9300
weighted: accuracy = 0.9400

CV 3-fold de votadores (accuracy)
majority: mean=0.9160  std=0.0049
    soft: mean=0.9240  std=0.0073
weighted: mean=0.9260  std=0.0100


<div align="center">
  
#  Cuestionario teórico

<img src="https://99px.ru/sstorage/86/2016/11/image_862911161940187734278.gif" width="280"/>


</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 1️⃣ ¿Qué ventajas ofrece combinar varios clasificadores mediante un sistema de votación?

Al combinar modelos, aprovechamos las fortalezas de cada uno y compensamos sus debilidades. Los errores de un modelo no se superponen completamente con los de otro, así que la decisión final es más robusta y estable. 

Además, reducimos la varianza (menos sobreajuste), mejoramos la generalización y protegemos contra malas predicciones individuales.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 2️⃣ Explica la diferencia conceptual entre votación dura y votación suave.

Votación dura: Cada modelo dice "fuerte" o "débil" y se cuentan los votos. La mayoría gana. No importa qué tan seguro esté cada modelo de su decisión.

Votación suave: Aquí cada modelo no solo dice su veredicto, sino qué tan seguro está (probabilidad de 0 a 1). Se promedian las probabilidades y se elige según el promedio. Es más refinado porque usa información adicional sobre la confianza de cada predictor.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 3️⃣ ¿Qué tipo de problemas puede presentar una votación por mayoría simple?

- Si varios modelos cometen los mismos errores (están correlacionados), la votación solo refuerza el error.
- Si incluyes muchos modelos mediocres, pueden opacar a uno muy bueno.
- En datasets desbalanceados, puede sesgar hacia la clase mayoritaria.

La clave es necesitar diversidad en el equipo de modelos.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 4️⃣ ¿En qué casos un ensamble puede tener peor desempeño que un modelo individual?

Un ensamble puede empeorar si:

1. Los modelos base son muy débiles.
2. Los errores están correlacionados - Si todos fallan de la misma manera, combinarlos no ayuda.
3. Incluyes modelos pésimos - Un modelo malo puede arrastrar al equipo hacia abajo, especialmente en votación dura.
4. Sobreajuste colectivo - Si todos los modelos se sobreajustaron de manera similar.

Lección: Más modelos ≠ mejor. Se necesita calidad y diversidad, no solo cantidad.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 5️⃣ ¿Qué papel juega la independencia de los errores de los clasificadores base en un ensamble?

La independencia de errores es crucial. Si los modelos cometen errores diferentes, al combinarlos esos errores se cancelan parcialmente.

Si tenemos $M$ modelos independientes con error $\epsilon$, el error del ensamble puede reducirse proporcionalmente.

En la práctica: Por eso Random Forest funciona tan bien: usa bootstrap (muestras aleatorias) y feature randomness (subconjuntos aleatorios de variables) para forzar diversidad entre árboles.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 6️⃣ ¿Cuál es la diferencia entre ensambles homogéneos y heterogéneos?

Ensambles homogéneos: Usan el mismo tipo de modelo base con diferentes configuraciones o datos (como Random Forest con muchos árboles de decisión). Ventaja: Fácil de paralelizar y optimizar.

Ensambles heterogéneos: Combinan diferentes tipos de modelos (KNN, SVM, árboles, etc.), como nuestro votador (KNN + Logistic + Tree + SVM). Ventaja: Mayor diversidad de estrategias.

¿Cuál es mejor? Depende. Heterogéneos suelen ser más diversos, pero homogéneos (como Random Forest) pueden ser muy potentes si se fuerza diversidad internamente.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 7️⃣ ¿Cómo afecta el número de modelos en el ensamble al rendimiento final?

Ley de rendimientos decrecientes: Agregar más modelos ayuda, pero el beneficio disminuye con cada adición.

- Pocos modelos (2-3): Beneficio significativo
- Cantidad moderada (5-10): Buena relación costo-beneficio
- Muchos modelos (20+): Poco beneficio adicional, más costo computacional

En la práctica: Es mejor tener 5 modelos muy diversos y buenos que 50 modelos similares.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 8️⃣ ¿Qué es el teorema del "no free lunch" y cómo se relaciona con los ensambles?

Teorema del "No Free Lunch": No existe un algoritmo que sea mejor que todos los demás en todos los problemas posibles.

Implicación: Cada algoritmo tiene sus fortalezas en ciertos tipos de datos. Un modelo que funciona perfecto para un problema puede ser pésimo para otro.

Relación con ensambles: Por eso los ensambles funcionan. Al combinar modelos con diferentes sesgos inductivos, cubrimos más casos. Un modelo puede ser mejor para:
- Datos lineales → Logistic Regression
- Datos con interacciones complejas → Decision Tree
- Datos con fronteras no lineales → SVM (kernel RBF)
- Datos locales/por vecindad → KNN

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 9️⃣ ¿Qué es el sesgo (bias) y la varianza en machine learning y cómo los ensambles ayudan a controlarlos?

Sesgo (Bias): Error por suposiciones simplificadas. Un modelo con alto sesgo es demasiado simple (underfitting). Usar una línea recta para datos circulares sería un ejemplo.

Varianza: Error por sensibilidad a los datos de entrenamiento. Un modelo con alta varianza cambia mucho con pequeños cambios en los datos (overfitting). Un árbol de decisión muy profundo que memoriza el training set sería un ejemplo.

Cómo ayudan los ensambles:

- Bagging (como Random Forest) → Reduce varianza promediando modelos entrenados en diferentes muestras bootstrap
- Boosting → Reduce sesgo entrenando modelos secuencialmente que corrigen errores previos
- Voting → Balance entre ambos, dependiendo de la diversidad de modelos

Trade-off: Generalmente, reducir uno aumenta el otro. Los ensambles ayudan a encontrar un mejor balance.

</div>

---

<div style="background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0;">

### 🔟 ¿Por qué Random Forest se considera un método de ensamble, y cómo se diferencia de nuestro sistema de votación?

Random Forest es un ensamble porque:

1. Combina múltiples árboles de decisión (típicamente 100-500)
2. Cada árbol se entrena en una muestra bootstrap (con reemplazo) del dataset
3. En cada split, solo considera un subconjunto aleatorio de features
4. La predicción final es votación mayoritaria (clasificación) o promedio (regresión)

Diferencias con nuestro sistema:

| Aspecto | Nuestro Voting | Random Forest |
|---------|---------------|---------------|
| Tipo de modelos | Heterogéneo (KNN, SVM, Logistic, Tree) | Homogéneo (solo árboles) |
| Número de modelos | 4 modelos | Típicamente 100-500 árboles |
| Fuente de diversidad | Diferentes algoritmos | Bootstrap + feature randomness |
| Costo computacional | Bajo (4 modelos) | Alto (cientos de árboles) |
| Tipo de ensamble | Voting (hard/soft/weighted) | Bagging |

¿Cuál es mejor? Depende del problema. Random Forest suele ser muy robusto, pero nuestro voting puede ser más interpretable y combinar las fortalezas de algoritmos muy diferentes.

</div>

---

<div align="center">
  <img src="https://i.gifer.com/embedded/download/8XbQ.gif" width="250"/>
  <p style="font-style: italic; color: #999; margin-top: 15px;">¡Fin del cuestionario! 🎉</p>
</div>


---

<div align="center">
  
# 🎓 Conclusiones del laboratorio

<img src="https://i.pinimg.com/originals/02/52/0b/02520bafe50166360f69ac620b22f614.gif" width="300"/>

</div>

---

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### 📊 1. Construcción de datasets balanceado

Aprendimos a usar muestreo estratificado para obtener un dataset representativo.

  </div>
</div>

---

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### 🤖 2. Entrenamiento de múltiples modelos

Probamos 9 modelos diferentes (KNN, Logistic Regression, Decision Tree, SVM, LDA, Ridge, SGD, Naive Bayes, Random Forest) y descubrimos sus fortalezas y debilidades.

Lección clave: Cada algoritmo tiene su especialidad.

  </div>
</div>

---

<div style="background: linear-gradient(135deg, #ff6b6b 0%, #feca57 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### 🤝 3. Poder de los ensambles

Vimos cómo combinar modelos mediante votación (Majority, Soft, Weighted) puede crear predictores más robustos y estables.

Conclusión: El trabajo en equipo supera el talento individual. Un ensamble bien diseñado casi siempre supera a sus miembros individuales.

  </div>
</div>

---

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### 🔍 4. Importancia de las features

Descubrimos que <code>base_experience</code> es la característica más importante para predecir el poder de un Pokémon, seguida de <code>weight</code> y <code>height</code>.

Insight: No todas las características son iguales. Identificar las más relevantes puede simplificar modelos sin perder rendimiento.

  </div>
</div>

---

<div style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### ⚖️ 5. Balance entre complejidad y desempeño

No siempre el modelo más complejo es el mejor. Los modelos lineales (Logistic Regression, LDA, Ridge) demostraron ser sorprendentemente efectivos.

Principio de parsimonia: "Keep it simple". Si un modelo simple funciona bien, no necesitas uno complejo que sea más difícil de interpretar y mantener.

  </div>
</div>

---

<div style="background: linear-gradient(135deg, #30cfd0 0%, #330867 100%); padding: 3px; border-radius: 12px; margin: 25px 0;">
  <div style="background: white; padding: 25px; border-radius: 10px;">

### 🎲 6. Importancia de la validación

Usar holdout 80/20 y cross-validation 3-fold nos permitió:
- Medir la estabilidad de los modelos
- Detectar overfitting
- Obtener estimaciones más confiables del rendimiento

Lección: Nunca confíes en una sola métrica o partición. La validación cruzada es esencial para evaluar modelos de forma robusta.

  </div>
</div>

---

<div style="text-align: center; padding: 30px; background: linear-gradient(135deg, #ffd89b 0%, #19547b 100%); color: white; border-radius: 15px; margin-top: 30px; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
  
<h1 style="margin: 0; font-size: 2.5em;">🏆 ¡ENTRENAMIENTO COMPLETADO! 🏆</h1>

<div style="margin: 20px 0;">
  <img src="https://99px.ru/sstorage/86/2016/11/image_862911161940187734278.gif" width="200"/>
</div>

<p style="font-size: 1.3em; margin: 15px 0;">Has dominado el arte de los ensambles de votación</p>


<h3 style="margin-top: 25px; color: #fff;">💤 Team Snorlax - Lab ML P7 💤</h3>

</div>
